In [2]:

from keras.datasets import mnist
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt


In [3]:
pip install opencv-python==3.4.2.16

In [4]:
pip install opencv-contrib-python==3.4.2.16

In [5]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [6]:
#edge extraction suing canny detector
def pre_processing(image):
  image = cv2.GaussianBlur(image.astype(np.uint8),(5,5),0)
  edge = cv2.Canny(image,50,50)
  return edge

In [6]:
sift = cv2.xfeatures2d.SIFT_create()
kp = sift.detect(edge,None)
kp, des = sift.compute(edge, kp)
img2 = cv2.drawKeypoints(edge,kp,color=(0,255,0),outImage=None, flags=0)
plt.imshow(img2)
plt.show()

NameError: ignored

In [7]:
#SIFT extraction

sift = cv2.xfeatures2d.SIFT_create()
temp = []
temp_y=[]
for j in range(x_train.shape[0]):
    e1 = pre_processing(x_train[j].astype(np.uint8))
    kx, des = sift.detectAndCompute(e1, None)
    if (des is not None):
      temp.append(des)
      temp_y.append(y_train[j])

In [8]:
sift = cv2.xfeatures2d.SIFT_create()
test = []
test_y=[]
for k in range(x_test.shape[0]):
    e1 = pre_processing(x_test[k].astype(np.uint8))
    kx, des = sift.detectAndCompute(e1, None)
    if (des is not None):
      test.append(des)
      test_y.append(y_test[k])

In [11]:

train_desc = []
for p in temp:
  for d in p:
    train_desc.append(d)

In [12]:
test_desc = []
for q in test:
  for d in q:
    test_desc.append(d)

In [13]:
#converting into np array
train_desc = np.array(train_desc)

test_desc = np.array(test_desc)

In [14]:
print(train_desc.shape)
print(test_desc.shape)

(176574, 128)
(29580, 128)


In [15]:
kclusters = 300
from sklearn.cluster import MiniBatchKMeans
cls = MiniBatchKMeans(kclusters)


cls.fit(train_desc)

MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
                init_size=None, max_iter=100, max_no_improvement=10,
                n_clusters=300, n_init=3, random_state=None,
                reassignment_ratio=0.01, tol=0.0, verbose=0)

In [19]:
#hist of features of each image
pred = []
for v in temp:
    pred.append(cls.predict(v))

# Creating feature vector for the images 
training_h = [np.bincount(feature,minlength=kclusters) for feature in pred]


training_h = np.array(training_h)

# Now we are going to make histogram of features for each image using cluster centroid found by DBSCAN
prediction_test = []
for v in test:
    prediction_test.append(cls.predict(v))

# Creating feature vector for the images from the predict cluster of features it is belonging to.
testing_h = [np.bincount(feature,minlength=kclusters) for feature in prediction_test]


testing_h = np.array(testing_h)


# Fitting an svm model on the feature vector created for each image

from sklearn.svm import SVC 
model = SVC(kernel='rbf')





In [ ]:
model.fit(training_h,temp_y)


In [1]:
print(model.score(test_hist,test_y))

NameError: ignored